In [121]:
from transformers import BertModel, BertTokenizer, AdamWeightDecay, get_linear_schedule_with_warmup
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd
from textwrap import wrap

In [122]:
# Inicialization of the BERT model
RANDOM_SEED = 42
MAX_LEN = 500
BATCH_SIZE = 16
DATASET_PATH = "historias_clinicas_procesadas.xlsx"
NCLASS = 6

np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


In [123]:
import torch
torch.cuda.is_available()

True

In [124]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.8


In [125]:
print("CUDA disponible:", torch.cuda.is_available())
print("Versión compilada con PyTorch:", torch.version.cuda)
print("Versión de cuDNN:", torch.backends.cudnn.version())
print("Nombre de GPU:", torch.cuda.get_device_name(0))

CUDA disponible: True
Versión compilada con PyTorch: 12.8
Versión de cuDNN: 91002
Nombre de GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [126]:
# Cargar dataset

df = pd.read_excel(DATASET_PATH)
df.head()


,sexo,edad,grupo,especialidad_medica,subjetivo,objetivo,concatenada,sexo_codificado,grupo_codificado
0,Femenino,38,Otros trastornos,PSICOLOGÍA,"Paciente refiere: ""Me empezaron a dar como uno...","Paciente alerta, colaboradora con apariencia o...","['empezar', 'episodio', 'tom', 'pastilla', 'do...",0,0
1,Masculino,22,T. externalizantes,PSICOLOGÍA,"Paciente refiere ""Me he sentido muy mal, en el...","Paciente a quien evaluó por primera vez, alert...","['sentido', 'trabajar', 'concentrar yo', 'cosa...",1,4
2,Masculino,9,Otros trastornos,PSICOLOGÍA,"La madre refiere ""el viene por un acompañamien...",NaN,"['madre', 'venir', 'acompañamiento', 'emociona...",1,0
3,Masculino,28,Otros trastornos,PSICOLOGÍA,"Paciente refiere ""Estas cosas que han pasado m...","Paciente quien evalúo por primera vez, alerta,...","['cosa', 'problema', 'empresa', 'ocasión', 'sa...",1,0
4,Femenino,8,Otros trastornos,PSICOLOGÍA,"La madre refiere ""Ella ha manifestado ciertas ...","Paciente ingresa en compañía de la madre, aler...","['madre', 'manifestar', 'conducta', 'palabrase...",0,0


In [127]:
# Probamos con 500 registros utilizando sample y Ramdom Seed
# df = df.sample(n=5000, random_state=RANDOM_SEED).reset_index(drop=True)

In [128]:
df.head()

,sexo,edad,grupo,especialidad_medica,subjetivo,objetivo,concatenada,sexo_codificado,grupo_codificado
0,Femenino,38,Otros trastornos,PSICOLOGÍA,"Paciente refiere: ""Me empezaron a dar como uno...","Paciente alerta, colaboradora con apariencia o...","['empezar', 'episodio', 'tom', 'pastilla', 'do...",0,0
1,Masculino,22,T. externalizantes,PSICOLOGÍA,"Paciente refiere ""Me he sentido muy mal, en el...","Paciente a quien evaluó por primera vez, alert...","['sentido', 'trabajar', 'concentrar yo', 'cosa...",1,4
2,Masculino,9,Otros trastornos,PSICOLOGÍA,"La madre refiere ""el viene por un acompañamien...",NaN,"['madre', 'venir', 'acompañamiento', 'emociona...",1,0
3,Masculino,28,Otros trastornos,PSICOLOGÍA,"Paciente refiere ""Estas cosas que han pasado m...","Paciente quien evalúo por primera vez, alerta,...","['cosa', 'problema', 'empresa', 'ocasión', 'sa...",1,0
4,Femenino,8,Otros trastornos,PSICOLOGÍA,"La madre refiere ""Ella ha manifestado ciertas ...","Paciente ingresa en compañía de la madre, aler...","['madre', 'manifestar', 'conducta', 'palabrase...",0,0


In [129]:
# Calcular la longitud máxima de las historias clínicas en términos de número de palabras
max_lenght = df["concatenada"].apply(lambda x: len(x.split())).max()

print(f"La longitud máxima de las historias clínicas es: {max_lenght} palabras.")

La longitud máxima de las historias clínicas es: 965 palabras.


In [130]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9809 entries, 0 to 9808
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   sexo                 9809 non-null   object
 1   edad                 9809 non-null   int64 
 2   grupo                9809 non-null   object
 3   especialidad_medica  9809 non-null   object
 4   subjetivo            9733 non-null   object
 5   objetivo             9774 non-null   object
 6   concatenada          9809 non-null   object
 7   sexo_codificado      9809 non-null   int64 
 8   grupo_codificado     9809 non-null   int64 
dtypes: int64(3), object(6)
memory usage: 689.8+ KB


In [131]:
df.head(10)

,sexo,edad,grupo,especialidad_medica,subjetivo,objetivo,concatenada,sexo_codificado,grupo_codificado
0,Femenino,38,Otros trastornos,PSICOLOGÍA,"Paciente refiere: ""Me empezaron a dar como uno...","Paciente alerta, colaboradora con apariencia o...","['empezar', 'episodio', 'tom', 'pastilla', 'do...",0,0
1,Masculino,22,T. externalizantes,PSICOLOGÍA,"Paciente refiere ""Me he sentido muy mal, en el...","Paciente a quien evaluó por primera vez, alert...","['sentido', 'trabajar', 'concentrar yo', 'cosa...",1,4
2,Masculino,9,Otros trastornos,PSICOLOGÍA,"La madre refiere ""el viene por un acompañamien...",NaN,"['madre', 'venir', 'acompañamiento', 'emociona...",1,0
3,Masculino,28,Otros trastornos,PSICOLOGÍA,"Paciente refiere ""Estas cosas que han pasado m...","Paciente quien evalúo por primera vez, alerta,...","['cosa', 'problema', 'empresa', 'ocasión', 'sa...",1,0
4,Femenino,8,Otros trastornos,PSICOLOGÍA,"La madre refiere ""Ella ha manifestado ciertas ...","Paciente ingresa en compañía de la madre, aler...","['madre', 'manifestar', 'conducta', 'palabrase...",0,0
5,Masculino,30,T. externalizantes,PSICOLOGÍA,"Paciente refiere ""yo creo como todo comienza d...","Paciente quien evalúo por primera vez, pacien...","['comenzar', 'crianza', 'mama', 'diagnostico',...",1,4
6,Femenino,60,Otros trastornos,PSICOLOGÍA,"Paciente refiere ""hace 3 años me separe, hace ...","Paciente quien evalúo por primera vez, pacient...","['año', 'separar', 'año', 'hija', 'tambin', 't...",0,0
7,Masculino,57,Otros trastornos,PSICOLOGÍA,"Paciente refiere ""Yo he sido muy introvertido,...","Paciente quien evalúo por primera vez, pacient...","['introvertido', 'expresar', 'sentimiento', 't...",1,0
8,Femenino,25,T. externalizantes,PSICOLOGÍA,"Hace un mes empece a sentir ansiedad, tuve com...","Paciente quien evalúo por primera vez, pacient...","['mes', 'empece', 'sentir', 'ansiedad', 'tener...",0,4
9,Femenino,40,Otros trastornos,PSICOLOGÍA,"Paciente refiere ""en los 15 de mi prima yo le ...","Paciente quien evalúo por primera vez, la cons...","['primar', 'llevar', 'comida', 'tiro', 'marco'...",0,0


In [132]:
# Removemos los corchetes
df["concatenada"] = df["concatenada"].apply(lambda x: str(x).replace("[","").replace("]",""))

# Removemos las comillas simples
df["concatenada"] = df["concatenada"].apply(lambda x: str(x).replace("'",""))

# Unimos las palabras separadas por comas
df["concatenada"] = df["concatenada"].apply(lambda x: str(x).replace(","," "))

In [133]:
df.head()

,sexo,edad,grupo,especialidad_medica,subjetivo,objetivo,concatenada,sexo_codificado,grupo_codificado
0,Femenino,38,Otros trastornos,PSICOLOGÍA,"Paciente refiere: ""Me empezaron a dar como uno...","Paciente alerta, colaboradora con apariencia o...",empezar episodio tom pastilla dormir tene...,0,0
1,Masculino,22,T. externalizantes,PSICOLOGÍA,"Paciente refiere ""Me he sentido muy mal, en el...","Paciente a quien evaluó por primera vez, alert...",sentido trabajar concentrar yo cosa vidato...,1,4
2,Masculino,9,Otros trastornos,PSICOLOGÍA,"La madre refiere ""el viene por un acompañamien...",NaN,madre venir acompañamiento emocional sepra...,1,0
3,Masculino,28,Otros trastornos,PSICOLOGÍA,"Paciente refiere ""Estas cosas que han pasado m...","Paciente quien evalúo por primera vez, alerta,...",cosa problema empresa ocasión saludar sal...,1,0
4,Femenino,8,Otros trastornos,PSICOLOGÍA,"La madre refiere ""Ella ha manifestado ciertas ...","Paciente ingresa en compañía de la madre, aler...",madre manifestar conducta palabrasello rel...,0,0


In [134]:
from transformers import BertTokenizer, BertModel

# TOKENIZACION

PRE_TRAINED_MODEL_NAME = "bert-base-multilingual-cased"
tokenizer = BertTokenizer.from_pretrained(PRE_TRAINED_MODEL_NAME)

In [135]:
# Ejemplos de tokenización

sample_text = "Hola soy un texto de ejemplo para la tokenización."
tokens = tokenizer.tokenize(sample_text)
tokens_ids = tokenizer.convert_tokens_to_ids(tokens)
print(f"Texto original: {sample_text}")
print(f"Tokens: {tokens}")
print(f"IDs de tokens: {tokens_ids}")

Texto original: Hola soy un texto de ejemplo para la tokenización.
Tokens: ['Ho', '##la', 'soy', 'un', 'texto', 'de', 'ejemplo', 'para', 'la', 'tok', '##eni', '##zación', '.']
IDs de tokens: [20220, 10330, 103559, 10119, 27888, 10104, 20223, 10220, 10109, 18436, 18687, 23700, 119]


In [136]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

2.9.0+cu128
True


In [137]:
# Instala PyTorch si no está instalado

# Codificacion para introducir a BERT

encoding = tokenizer.encode_plus(
    sample_text,
    add_special_tokens=True,
    max_length=MAX_LEN,
    return_token_type_ids=False,
    padding="max_length",
    truncation=True,
    return_attention_mask=True,
    return_tensors="pt",
)

In [138]:
encoding.keys()

KeysView({'input_ids': tensor([[   101,  20220,  10330, 103559,  10119,  27888,  10104,  20223,  10220,
          10109,  18436,  18687,  23700,    119,    102,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
     

In [139]:
print(tokenizer.convert_ids_to_tokens(encoding["input_ids"][0]))
print(encoding["input_ids"][0])
print(encoding["attention_mask"][0])

['[CLS]', 'Ho', '##la', 'soy', 'un', 'texto', 'de', 'ejemplo', 'para', 'la', 'tok', '##eni', '##zación', '.', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]',

In [140]:
# CREACION DEL DATASET

class CustomDataset(Dataset):
    def __init__(self, text, labels, tokenizer, max_len):
        self.text = text
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
      return len(self.text)

    def __getitem__(self, item):
      text = str(self.text[item])
      label = self.labels[item]
      encoding = tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_len,
          return_token_type_ids=False,
          padding="max_length",
          truncation=True,
          return_attention_mask=True,
          return_tensors="pt",
      )
      return {"text": text,
                "input_ids": encoding["input_ids"].flatten(),
                "attention_mask": encoding["attention_mask"].flatten(),
                "labels": torch.tensor(label, dtype=torch.long)}


In [141]:
df.columns

Index(['sexo', 'edad', 'grupo', 'especialidad_medica', 'subjetivo', 'objetivo',
       'concatenada', 'sexo_codificado', 'grupo_codificado'],
      dtype='object')

In [142]:
# Data Loader:

def data_loader(df, tokenizer, max_len, batch_size):
    dataset = CustomDataset(
        text = df["concatenada"].to_numpy(),
        labels = df["grupo_codificado"].to_numpy(),
        tokenizer = tokenizer,
        max_len = MAX_LEN
    )
    return DataLoader(dataset, batch_size = BATCH_SIZE, pin_memory=True)


In [143]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=RANDOM_SEED)

train_data_loader = data_loader(df_train, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = data_loader(df_test, tokenizer, MAX_LEN, BATCH_SIZE)

In [144]:
# Red Neuronal para el modelo BERT

class BERTTextClassifier(nn.Module):
    def __init__(self, n_classes):
        super(BERTTextClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
      outputs = self.bert(
          input_ids = input_ids,
          attention_mask = attention_mask
      )
      pooled_out = outputs.pooler_output
      drop_out = self.dropout(pooled_out)
      output = self.linear(drop_out)
      return output


In [145]:
model = BERTTextClassifier(NCLASS)
model = model.to(device)

In [146]:
print(model)

BERTTextClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, eleme

In [147]:
# ENTRENAMIENTO

EPOCHS = 5
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
# total_steps = (train_data_loader.batch_size) * EPOCHS
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = total_steps
)

loss_fn = nn.CrossEntropyLoss().to(device)

In [148]:
# Iteracion de Entranamiento

def train_model(model, data_loader, loss_fn, optimizer, device, scheduler, n_examples):
    model = model.train()
    losses = []
    correct_predictions = 0
    for batch in data_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        outputs = model(input_ids = input_ids, attention_mask = attention_mask)
        _, preds = torch.max(outputs, dim=1)
        loss = loss_fn(outputs, labels)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
    return correct_predictions.double() / n_examples, np.mean(losses)

def eval_model(model, data_loader, loss_fn, device, n_examples):
    model = model.eval()
    losses = []
    correct_predictions = 0
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            outputs = model(input_ids = input_ids, attention_mask = attention_mask)
            _, preds = torch.max(outputs, dim=1)
            loss = loss_fn(outputs, labels)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())
    return correct_predictions.double() / n_examples, np.mean(losses)

In [149]:
len(df_train)

7847

In [150]:
import time
from tqdm.notebook import tqdm

# Entrenamiento del Modelo con barras de progreso
for epoch in range(EPOCHS):
  print("Epoch {} de {}".format(epoch + 1, EPOCHS))
  print("-----------------------------------")
  start_time = time.time()

  # --- Training loop with tqdm ---
  model.train()
  train_losses = []
  train_correct = 0
  train_total = len(df_train)

  train_iter = tqdm(train_data_loader, desc=f"Epoch {epoch+1}/{EPOCHS} - Train", leave=False)
  for batch in train_iter:
    input_ids = batch["input_ids"].to(device)
    attention_mask = batch["attention_mask"].to(device)
    labels = batch["labels"].to(device)

    outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    _, preds = torch.max(outputs, dim=1)
    loss = loss_fn(outputs, labels)

    train_correct += torch.sum(preds == labels)
    train_losses.append(loss.item())

    loss.backward()
    nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()
    scheduler.step()
    optimizer.zero_grad()

    train_iter.set_postfix(loss=np.mean(train_losses), acc=(train_correct.double() / train_total).item())

  train_acc = train_correct.double() / train_total
  train_loss = np.mean(train_losses)

  # --- Evaluation loop with tqdm ---
  model.eval()
  test_losses = []
  test_correct = 0
  test_total = len(df_test)

  eval_iter = tqdm(test_data_loader, desc=f"Epoch {epoch+1}/{EPOCHS} - Eval", leave=False)
  with torch.no_grad():
    for batch in eval_iter:
      input_ids = batch["input_ids"].to(device)
      attention_mask = batch["attention_mask"].to(device)
      labels = batch["labels"].to(device)

      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      _, preds = torch.max(outputs, dim=1)
      loss = loss_fn(outputs, labels)

      test_correct += torch.sum(preds == labels)
      test_losses.append(loss.item())

      eval_iter.set_postfix(loss=np.mean(test_losses), acc=(test_correct.double() / test_total).item())

  test_acc = test_correct.double() / test_total
  test_loss = np.mean(test_losses)

  end_time = time.time()

  print("Entramiento: Loss: {:.4f}, accuracy: {:.4f}".format(train_loss, train_acc))
  print("Evaluacion: Loss: {:.4f}, accuracy: {:.4f}".format(test_loss, test_acc))
  print(f"Tiempo de la época: {end_time - start_time:.2f} segundos")
  print("")


Epoch 1 de 5
-----------------------------------


Epoch 1/5 - Train:   0%|          | 0/491 [00:00<?, ?it/s]

Epoch 1/5 - Eval:   0%|          | 0/123 [00:00<?, ?it/s]

Entramiento: Loss: 1.5323, accuracy: 0.3689
Evaluacion: Loss: 1.4924, accuracy: 0.3889
Tiempo de la época: 1660.91 segundos

Epoch 2 de 5
-----------------------------------


Epoch 2/5 - Train:   0%|          | 0/491 [00:00<?, ?it/s]

...

Epoch 4/5 - Eval:   0%|          | 0/123 [00:00<?, ?it/s]

Entramiento: Loss: 1.0216, accuracy: 0.6275
Evaluacion: Loss: 1.2322, accuracy: 0.5566
Tiempo de la época: 1630.41 segundos

Epoch 5 de 5
-----------------------------------


Epoch 5/5 - Train:   0%|          | 0/491 [00:00<?, ?it/s]

Epoch 5/5 - Eval:   0%|          | 0/123 [00:00<?, ?it/s]

Entramiento: Loss: 0.8996, accuracy: 0.6812
Evaluacion: Loss: 1.2323, accuracy: 0.5596
Tiempo de la época: 1630.03 segundos

